In [15]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from local_settings import postgresql as settings

In [16]:
import os
print(os.getcwd())

d:\Daniel\data_science\portfolio_projects\food_carbon_fp


In [17]:
def get_engine(user, passwd, host, port, db):
    url = f"mssql+pyodbc://{user}:{passwd}@{host},{port}/{db}?driver=ODBC+Driver+17+for+SQL+Server"
    engine = create_engine(url, pool_size= 50, echo= False)

    return engine

In [18]:
engine = get_engine(
    user= settings['pguser'],
    passwd=settings['pgpasswd'],
    host=settings['pghost'],
    port=settings['pgport'],
    db=settings['pgdb']
)

connection = engine.connect()

In [19]:
from sqlalchemy import inspect, MetaData, Table


inspector = inspect(engine)
print(inspector.get_table_names())

['food_cfp', 'food_cfp_2018']


In [20]:
metadata = MetaData()

carbonfp_df = Table('food_cfp_2018', metadata, autoload= True, autoload_with= engine)

print(carbonfp_df.columns.keys())

['Country', 'Product', 'Metric', 'Value']


In [21]:
from sqlalchemy import select
import pandas as pd

stmt = select([carbonfp_df])

results = connection.execute(stmt).fetchmany(size=10)

df = pd.DataFrame(results)
df.columns = results[0].keys()

df

,Country,Product,Metric,Value
0,Albania,Milk - inc. Cheese,Kg CO2/person/year,432.619995
1,Albania,Pork,Kg CO2/person/year,38.509998
2,Albania,Fish,Supplied for Consumption (kg/person/year),12.450000
3,Albania,Soybeans,Kg CO2/person/year,0.000000
4,Albania,Fish,Kg CO2/person/year,6.150000
5,Albania,Rice,Kg CO2/person/year,9.960000
6,Albania,Fish,Supplied for Consumption (kg/person/year),3.850000
7,Albania,Milk - inc. Cheese,Supplied for Consumption (kg/person/year),303.720001
8,Albania,Nuts inc. Peanut Butter,Supplied for Consumption (kg/person/year),4.360000
9,Albania,Lamb & Goat,Kg CO2/person/year,536.500000


##### Exploratory Data Analysis

In [43]:
# Create a function for converting and printing results with pandas

def results_pandas(stmt):
    return pd.DataFrame(connection.execute(stmt))

In [49]:
# Evaluated products and metrics

stmt = select(carbonfp_df.columns.Product.distinct(), carbonfp_df.columns.Metric)
stmt = stmt.order_by(carbonfp_df.columns.Product)

results_pandas(stmt)

,0,1
0,Beef,Supplied for Consumption (kg/person/year)
1,Beef,Kg CO2/person/year
2,Difference between Animal Products/person/year...,Difference between Animal Products/person/year...
3,Eggs,Kg CO2/person/year
4,Fish,Kg CO2/person/year
5,Fish,Supplied for Consumption (kg/person/year)
6,Lamb & Goat,Supplied for Consumption (kg/person/year)
7,Lamb & Goat,Kg CO2/person/year
8,Milk - inc. Cheese,Kg CO2/person/year
9,Milk - inc. Cheese,Supplied for Consumption (kg/person/year)
